# 1. Import Modules

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import datetime

# 2. Read Data

In [2]:
df_bw_file = pd.read_csv('SW_BusinessWeekly_KeyWords.csv')

In [3]:
df_bw_file

URL
0  https://www.businessweekly.com.tw/business/blo...
1  https://www.businessweekly.com.tw/focus/blog/3...
2  https://www.businessweekly.com.tw/focus/blog/3...
3  https://www.businessweekly.com.tw/focus/blog/3...
4  https://www.businessweekly.com.tw/focus/blog/3...
5  https://www.businessweekly.com.tw/business/blo...
6  https://www.businessweekly.com.tw/careers/blog...
7  https://www.businessweekly.com.tw/focus/blog/3...
8  https://www.businessweekly.com.tw/focus/blog/3...
9  https://www.businessweekly.com.tw/internationa...

# 3. Start to Scrape and Export Data

In [4]:
url_number = 0
today_date = datetime.date.today()
content_id = [] # 這邊不會用到
df_bw_sw_all = pd.DataFrame()

bw_column_name = ['content_id'
                 ,'content_title_h1'
                 ,'content_title_h2'
                 ,'content_class_main'
                 ,'content_class_sub'
                 ,'content_date_time'
                 ,'content_all'
                 ,'content_tag']

for URL in df_bw_file['URL']:
    
    content_id = [] # 基本上不會用到，如果是舊版網頁，此欄位先放文章的url，以便之後查詢
    
    try:
        time.sleep(5)
        url = URL
        print(url)
        headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
        response = requests.get(url, headers = headers)
        bw_html = BeautifulSoup(response.text, "lxml")
        
        # --- Begin 大標題 ---

        lst_title_h1 = []
        title_h1 = bw_html.find_all("h1")  # 搜尋所有h1
        for title in title_h1:
            print(title.getText())  # 只提取文字部分
            lst_title_h1.append(title.getText())

        # --- End 大標題 ---
        
        # --- Begin 次標題 ---

        lst_title_h2 = []
        title_h2 = bw_html.find_all("h2")  # 搜尋所有h2
        for title in title_h2:
            print(title.getText())  # 只提取文字部分
            lst_title_h2.append(title.getText())

        # --- End 次標題 ---
        
        # --- Begin 文章分類 Main and sub ---

        lst_content_class_main = []
        lst_content_class_sub = []
        content_class_main = bw_html.find_all("li", {"class":"breadcrumb-item"}) 

        class_num = 0

        for item in content_class_main:
            if class_num == 1: # 1是主標題
                lst_content_class_main.append(item.getText())
                print(item.getText())
            elif class_num == 2: # 2是次標題
                lst_content_class_sub.append(item.getText())
                print(item.getText())

            class_num += 1

        # --- End 文章分類 Main and sub ---

        content_all = bw_html.find_all("div", {"id":"divbody"})  # 文章內容
        
        # --- Begin 文章發佈日 ---

        lst_content_date = []

        for content in content_all:
            ct_date  = content.find("div", {"id":"AppGetDate"})  # 文章發佈日
            lst_content_date.append(ct_date.getText()[:10])
            print("文章發布日:", ct_date.getText()[:10])

        # --- End 文章分類及文章發佈日 ---
        
        # --- Begin 前言 ---
        
        ## 改成 meta description

        lst_content_introduction = []
        
        ct_introduction = bw_html.find("meta", {"name":"description"}).get("content")
        print("meta description: ", ct_introduction)
        lst_content_introduction.append(ct_introduction)

#         for content in content_all:  # r: 一個li區塊
#             ct_introduction = content.find("div", {"class":"Single-summary-content WebAbstract"})  # 文章簡介
#             lst_content_introduction.append(ct_introduction.getText())
#             print(ct_introduction.getText())

        # --- End 前言 ---
        
        # --- Begin 內文 ---

        lst_content_all = []

        for content in content_all:
            ct_all = content.find("div", {"class":"Single-article WebContent"})
            lst_content_all.append(ct_all.getText())
    #         print(ct_all.getText())

        # --- End 內文 ---
        
        # --- Begin 標籤 ---

        lst_content_tag_temp = []
        lst_content_tag = []
        content_tag = bw_html.find_all("section", {"class":"Single-tag-list d-xs-flex"})  # 標籤
        tag_num = 0

        for tag in content_tag:  # r: 一個li區塊
            lst_content_tag_temp.append(tag.getText())
            lst_content_tag_temp = [tag_temp.split('\n') for tag_temp in lst_content_tag_temp]
            print(tag.getText())

        for tag_num in range(1, len(lst_content_tag_temp[0]) - 1):
            lst_content_tag.append(lst_content_tag_temp[0][tag_num])

        # --- End 標籤 ---
        
        # List 轉 Series
        series_id = pd.Series(content_id)
        series_title_h1 = pd.Series(lst_title_h1)

        lst_title_h2 = ",".join(lst_title_h2) # 把list中的資料用","join起來
        series_title_h2 = pd.Series(lst_title_h2)

        series_content_class_main = pd.Series(lst_content_class_main)
        series_content_class_sub = pd.Series(lst_content_class_sub)
        series_content_date = pd.Series(lst_content_date)
        series_content_all = pd.Series(lst_content_all)

        lst_content_tag = ",".join(lst_content_tag) # 把list中的資料用","join起來
        series_content_tag = pd.Series(lst_content_tag)
        
#         bw_column_name = ['content_id'
#                          ,'content_title_h1'
#                          ,'content_title_h2'
#                          ,'content_class_main'
#                          ,'content_class_sub'
#                          ,'content_date_time'
#                          ,'content_all'
#                          ,'content_tag']

        # Series 放到 DataFrame
        df_bw_sw_keyword = pd.concat([series_id
                                     ,series_title_h1
                                     ,series_title_h2
                                     ,series_content_class_main
                                     ,series_content_class_sub
                                     ,series_content_date
                                     ,series_content_all
                                     ,series_content_tag]
                                     ,axis = 1)
        
        df_bw_sw_keyword.columns = bw_column_name

        url_number += 1
        
        df_bw_sw_all = pd.concat([df_bw_sw_all, df_bw_sw_keyword], ignore_index=True)
        
    except:
        print("Manul:", url)
        
        content_id.append(url)
        print("content_id", content_id)
        
        series_id = pd.Series(content_id)
        
        df_bw_sw_keyword_except = pd.concat([series_id], axis = 1)
        
        df_bw_sw_keyword_except.columns = ["content_id"]
        df_bw_sw_all = pd.concat([df_bw_sw_all, df_bw_sw_keyword_except], ignore_index=True)
        
        url_number += 1
    
print("----------------- Finish -------------------")

df_bw_sw_all.to_csv(f'{today_date}_BusinessWeekly_SimilarWeb_Keyword.csv', index = False, encoding = 'UTF-8-sig')


https://www.businessweekly.com.tw/business/blog/3014691
元大首推月配息ETF 00940、發行價10元，該買嗎？適合誰？
00940介紹
00940成分股：10大持股
00940基本檔案
00940特色
財經
投資理財
文章發布日: 2024/02/07
meta description:  元大投信表示，3月11日開始募集的ETF「元大台灣價值高息00940」，是一檔採用巴菲特價值投資理念的ETF，發行價10元，並採月配息，非常適合小資族投資。本文介紹00940的成分股及管理費等資訊。

收益平準金
ETF
月配息
00940



C:\Users\JoeJian\AppData\Local\Temp\ipykernel_18740\3153724512.py:126: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://www.businessweekly.com.tw/focus/blog/3014663
2024龍年吉祥話成語、創意諧音大全！祝福生意對象賀詞有哪些？
龍年成語
龍年4字吉祥話
龍年8字吉祥話
龍年搞笑、諧音吉祥話
萬用祝福吉祥話
祝賀常用的英文句子
2024年要小心說的吉祥話
焦點
時事分析
文章發布日: 2024/02/12
meta description:  過年見到許久未見的親朋好友，除了說一句「龍Time No See」、新年快樂、大吉大利，有哪些和龍有關的成語？或是龍年吉祥話和祝福語呢？不管要祝福長輩、平輩、晚輩或職場夥伴，本文都有相對應的吉祥話！

成語
過年
吉祥話
龍年
春節



C:\Users\JoeJian\AppData\Local\Temp\ipykernel_18740\3153724512.py:126: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://www.businessweekly.com.tw/focus/blog/3012795
新青年安心成家房貸懶人包！小資族靠它買房小心3大風險
「青年安心成家購屋優惠貸款精進方案」是什麼？
青年房貸新方案怎麼申請？
青年安心成家條件與申請資格
青年安心成家貸款新方案何時實施？
青年安心成家貸款新方案有哪些新增優惠？
青年安心成家貸款新方案利息怎麼計算？
青年安心成家貸款新方案注意事項，有3大風險
青年安心成家購屋優惠貸款問與答
焦點
時事分析
文章發布日: 2023/11/28
meta description:  行政院財政部推出「青年安心成家購屋優惠貸款精進方案」，8月1日上路，拉高青年購屋貸款額度、貸款期限，並延長還款寬限期、提高政府補貼貸款利率。申請青年房貸，只需年滿18歲、名下沒有住宅，就有申請資格。 

首購族
房貸補助
財政部
公股銀行
青安房貸
青年購屋
青安貸款



...

C:\Users\JoeJian\AppData\Local\Temp\ipykernel_18740\3153724512.py:126: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://www.businessweekly.com.tw/focus/blog/3014636
過年前包裹最晚哪天寄出？郵局銀行春節營業時間、ATM提款上限一次看
2024過年郵局休幾天？
過年郵局有送嗎？
郵局過年期間可以寄信嗎？
2024各家銀行過年營業時間
ATM提款上限、轉帳上限
焦點
時事分析
文章發布日: 2024/02/01
meta description:  農曆春節年假即將到來，你準備好紅包、換好新鈔了嗎？過年期間，錢不夠用要去哪裡領？如果突然有信件或包裹要寄送怎麼辦？郵局、銀行春節連假時有營業嗎？年假期間，郵局窗口停止服務，快捷郵件仍照常收送。銀行雖然

ATM
春節
郵局
過年
銀行



C:\Users\JoeJian\AppData\Local\Temp\ipykernel_18740\3153724512.py:126: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://www.businessweekly.com.tw/international/blog/3011742
永久居留日本，最新方案來了！只要住1年再加上「2條件」
國際
全球話題
文章發布日: 2023/02/18
meta description:  日本政府17日召開有關吸納外國人才的相關閣僚會議，為吸引更多在關鍵知識或技術方面優秀的海外人才，決定新設吸納制度。目前「高度專門職」居留資格的取得條件複雜，日本將新設「特別高度人才制度」，核准發給居留

日本
松野博一
居留權
移民

----------------- Finish -------------------


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_18740\3153724512.py:126: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


In [5]:
df_bw_sw_all

content_id                   content_title_h1  \
0         NaN   元大首推月配息ETF 00940、發行價10元，該買嗎？適合誰？   
1         NaN    2024龍年吉祥話成語、創意諧音大全！祝福生意對象賀詞有哪些？   
2         NaN         新青年安心成家房貸懶人包！小資族靠它買房小心3大風險   
3         NaN  過年前包裹最晚哪天寄出？郵局銀行春節營業時間、ATM提款上限一次看   
4         NaN    2024龍年吉祥話成語、創意諧音大全！祝福生意對象賀詞有哪些？   
5         NaN        過年股市連休9天！封關日是哪一天？2024年台股行事曆   
6         NaN   當親友過世時，英文除了"R.I.P"...10種慰問的英文這樣說   
7         NaN         新青年安心成家房貸懶人包！小資族靠它買房小心3大風險   
8         NaN  過年前包裹最晚哪天寄出？郵局銀行春節營業時間、ATM提款上限一次看   
9         NaN        永久居留日本，最新方案來了！只要住1年再加上「2條件」   

                                    content_title_h2 content_class_main  \
0           00940介紹,00940成分股：10大持股,00940基本檔案,00940特色                 財經   
1  龍年成語,龍年4字吉祥話,龍年8字吉祥話,龍年搞笑、諧音吉祥話,萬用祝福吉祥話,祝賀常用的英...                 焦點   
2  「青年安心成家購屋優惠貸款精進方案」是什麼？,青年房貸新方案怎麼申請？,青年安心成家條件與申...                 焦點   
3  2024過年郵局休幾天？,過年郵局有送嗎？,郵局過年期間可以寄信嗎？,2024各家銀行過年營...                 焦點   
4  龍年成語,龍年4字吉祥話,龍年8字吉祥話,龍年搞笑、諧音吉祥話,萬用祝福吉祥話,祝賀常用的英...                 焦點   
5  封關日當天可以下單嗎？,2024年股市何時封關？,2024年過年股市休幾天？,股市幾點可以下...                 財經   
6                                                                    職場   
7  「青年安心成家購屋優惠貸款精進方案」是什麼？,青年房貸新方案怎麼申請？,青年安心成家條件與申...                 焦點   
8  2024過年郵局休幾天？,過年郵局有送嗎？,郵局過年期間可以寄信嗎？,2024各家銀行過年營...                 焦點   
9                                                                    國際   

  content_class_sub content_date_time  \
0              投資理財        2024/02/07   
1              時事分析        2024/02/12   
2              時事分析        2023/11/28   
3              時事分析        2024/02/01   
4              時事分析        2024/02/12   
5              投資理財        2024/01/29   
6              英文學習        2017/07/19   
7              時事分析        2023/11/28   
8              時事分析        2024/02/01   
9              全球話題        2023/02/18   

                                         content_all  \
0  元大投信表示，3月11日開始募集的ETF「元大台灣價值高息00940」，是一檔採用巴菲特價值...   
1  👉過年紅包包多少｜新鈔哪裡換｜過年上班薪水算法｜大掃除禁忌｜過年郵局&銀行營業時間今年過年見...   
2  財政部推出「青年安心成家購屋優惠貸款精進方案」，拉高青年購屋貸款額度、貸款期限，並延長還款寬...   
3  👉過年紅包包多少｜新鈔哪裡換｜過年上班薪水算法｜大掃除禁忌農曆春節年假即將到來，你準備好紅包...   
4  👉過年紅包包多少｜新鈔哪裡換｜過年上班薪水算法｜大掃除禁忌｜過年郵局&銀行營業時間今年過年見...   
5  2024年農曆春節連假即將在2月8日開始，一連放7天、2月15日開工，台股卻將休市9天？為什...   
6  身邊的人若正經歷失去親人朋友、身體病痛，或是你只是純粹想表達你的支持，除了"I‘m sorr...   
7  財政部推出「青年安心成家購屋優惠貸款精進方案」，拉高青年購屋貸款額度、貸款期限，並延長還款寬...   
8  👉過年紅包包多少｜新鈔哪裡換｜過年上班薪水算法｜大掃除禁忌農曆春節年假即將到來，你準備好紅包...   
9  日本政府17日表示，將新設延攬外國菁英的新政策，有碩士學位且年收逾2000萬日圓（約新台幣4...   

                        content_tag  
0               收益平準金,ETF,月配息,00940  
1                   成語,過年,吉祥話,龍年,春節  
2  首購族,房貸補助,財政部,公股銀行,青安房貸,青年購屋,青安貸款  
3                   ATM,春節,郵局,過年,銀行  
4                   成語,過年,吉祥話,龍年,春節  
5                    股市,台股,過年,封關,休市  
6                rip正確用法,慰問英文,rip用法  
7  首購族,房貸補助,財政部,公股銀行,青安房貸,青年購屋,青安貸款  
8                   ATM,春節,郵局,過年,銀行  
9                    日本,松野博一,居留權,移民

In [6]:
# df_bw_sw_keyword = pd.concat([series_title_h1], axis = 1)
        
# df_bw_sw_keyword.columns = ["content_title_h1"]

In [ ]:
# df_bw_sw_keyword

In [ ]:
# df_bw_sw_all = pd.concat([df_bw_sw_all, df_bw_sw_keyword], ignore_index=True)

In [ ]:
# df_bw_sw_all